## import libraries

In [1]:
# add the path to my spark(you may not need these lines): 
import sys
sys.path.append('/home/victor/Downloads/spark-2.3.1-bin-hadoop2.7/python')
sys.path.append('/home/victor/Desktop/temp/TILES/src/pyspark_scripts')

In [2]:
from loadCSV import *
from MLModels import MLModel
from PySparkDFGetWindows import getWindows
from computeStats import computeStats
from datetime import timedelta, datetime
import pyspark
import os
from shutil import copyfile
%load_ext autoreload
%autoreload 2

## initialized pyspark

In [4]:
# Init Spark
sqlContext = initSpark()

## data path

In [5]:
filePath = '/home/victor/Desktop/'
tempPath = '/home/victor/Desktop/interData'
if not os.path.isdir(tempPath):
    os.makedirs(tempPath)

## load in data

In [6]:
idKeyData = CsvToDF(filePath + 'idKey.csv')
idKeyData.df.printSchema()

root
 |-- uid: string (nullable = true)
 |-- Activity_Feed_opened: integer (nullable = true)
 |-- App_started: integer (nullable = true)
 |-- Contact_opened: integer (nullable = true)
 |-- FAQ_opened: integer (nullable = true)
 |-- Facebook_Auth: integer (nullable = true)
 |-- Fitbit_Auth: integer (nullable = true)
 |-- Instagram_Auth: integer (nullable = true)
 |-- Internal_survey_opened: integer (nullable = true)
 |-- Push_instructions_opened: integer (nullable = true)
 |-- QR_code_opened: integer (nullable = true)
 |-- Regular_push_opened: integer (nullable = true)



In [7]:
dateKeyData = CsvToDF(filePath + 'dateKey.csv')
# drop column date
dateKeyData = dateKeyData.df.drop('Timestamp')
dateKeyData.printSchema()

root
 |-- Cardio_caloriesOut: double (nullable = true)
 |-- Cardio_max: double (nullable = true)
 |-- Cardio_min: double (nullable = true)
 |-- Cardio_minutes: double (nullable = true)
 |-- Fat Burn_caloriesOut: double (nullable = true)
 |-- Fat Burn_max: double (nullable = true)
 |-- Fat Burn_min: double (nullable = true)
 |-- Fat Burn_minutes: double (nullable = true)
 |-- NumberSteps: integer (nullable = true)
 |-- Out of Range_caloriesOut: double (nullable = true)
 |-- Out of Range_max: integer (nullable = true)
 |-- Out of Range_min: integer (nullable = true)
 |-- Out of Range_minutes: double (nullable = true)
 |-- Peak_caloriesOut: double (nullable = true)
 |-- Peak_max: double (nullable = true)
 |-- Peak_min: double (nullable = true)
 |-- Peak_minutes: double (nullable = true)
 |-- RestingHeartRate: double (nullable = true)
 |-- Sleep1BeginTimestamp: timestamp (nullable = true)
 |-- Sleep1Efficiency: double (nullable = true)
 |-- Sleep1EndTimestamp: timestamp (nullable = true)
 

In [8]:
timestampData = CsvToDF(filePath + 'timestampKey.csv')
timestampData.df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- NearestProximity: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- SurveyInAVehicle: integer (nullable = true)
 |-- surveyAtWork: integer (nullable = true)
 |-- psycCapPSYCAPScore: double (nullable = true)
 |-- workBeforeSurvey: integer (nullable = true)
 |-- psycCapHindranceStressorScore: double (nullable = true)
 |-- educationBeforeSurvey: integer (nullable = true)
 |-- surveyAtHome: integer (nullable = true)
 |-- psycFlexPositiveEmotionasBeforeSurvey: double (nullable = true)
 |-- personalActivityBeforeSurvey: integer (nullable = true)
 |-- psycFlexScore: double (nullable = true)
 |-- psycCapWorkEngagementScore: double (nullable = true)
 |-- durationInSeconds: integer (nullable = true)
 |-- surveyType: string (nullable = true)
 |-- onPhoneBeforeSurvey: integer (nullable = true)
 |-- familyBeforeSurvey: integer (nullable = true)
 |-- householingBeforeSurvey: integer (nullable =

## get window statistics

In [9]:
## since the timestamp dataframe is sparse, we need to get a list of timestamps where MGT_stress is not null

## input parameter should be a pyspark dataframe 
## returns a dataframe
def getValidateLableRows (dataframe, label):
    validRows = dataframe.filter( label + ' is not Null').select(['stress_mgt', 'date', 'Timestamp', 'ID'])
    return validRows
    
labelDF = getValidateLableRows(timestampData.df, 'stress_mgt')

In [10]:
def computeStats(df, gtCol, idCol, timeCol, colsToAggregate = None, stats = None, quantiles = False ):
    # get the dtype of ground truth variable
    dTypeGT = "double" #dict(dfGT.dtypes)[gtCol]
    if colsToAggregate == None:
        colsToAggregate = list(set(df.columns) - {"Timestamp", "date","ID"})
    # Convert the cols dtype to double
    for col in colsToAggregate:
        df = df.withColumn(col, df[col].cast("double"))
    if stats == None:        
        # Create an empty list of stats
        stats = []    
        for col in colsToAggregate:
            stats.append(F.count(col).alias(col+"Count"))
            stats.append(F.avg(col).alias(col+"Avg"))
            stats.append(F.min(col).alias(col+"Min"))
            stats.append(F.max(col).alias(col+"Max"))
            stats.append(F.stddev(col).alias(col+"Stddev"))
            stats.append(F.variance(col).alias(col+"Var"))
    # Start a timer for whole task
    t1 = time.time()
    print("Processing started at: ", int(t1), str(datetime.now()))
    dfAgg = df.agg(*stats)   
    dfAgg = dfAgg.withColumn('label', lit(gtCol).cast(dTypeGT))
    dfAgg = dfAgg.withColumn('ID', lit(idCol))
    dfAgg = dfAgg.withColumn('Timestamp', lit(timeCol))
    del(df)
    print("Time taken: ", int(time.time() -t1))
    return(dfAgg)

In [11]:
import pyspark.sql.functions as F

def getWindows(featureDF, labelDF, start = -10, end = 0, idColumnNameInLabel = 'ID', idColumnNameInFeature = 'ID', 
                        timeColumnNameInLabel = 'Timestamp', timeColumnNameInFeature = 'Timestamp', labelColumnName = 'stress_mgt'):
    # re-partition for better performance
    featureDF = featureDF.repartition(1000)
    featureDF.persist()
    i = 0
    
    # collect labelDF for fast processing
    GTRows = labelDF.collect()
    # find the corresponding window
    for row in GTRows:
        i += 1
        print (i/len(GTRows)*100)
        timestamp, participantID, label = row[timeColumnNameInLabel], row[idColumnNameInLabel], row[labelColumnName]
        windowStart = timestamp + timedelta(hours = start)
        windowEnd = timestamp + timedelta(hours = end)
        # filter conditions
        cond1 = featureDF[idColumnNameInFeature] == participantID
        cond2 = featureDF[timeColumnNameInFeature] >= windowStart
        cond3 = featureDF[timeColumnNameInFeature] <= windowEnd
        # Filter the dataframe
        dfWindow = featureDF.filter(cond1 & cond2 & cond3)
        # get window statistics
        dfWindow = computeStats(df = dfWindow, gtCol = label, idCol= participantID, timeCol= timestamp )
        #dfWindow.repartition(1).write.mode('overwrite').csv(tempPath + '/dfStats_stats/'+str(participantID)+'/'+str(i), header="true")
        if i == 1:
            stats = dfWindow
        else:
            stats = stats.union(dfWindow)
    stats.repartition(1).write.mode('overwrite').csv(tempPath + '/dfStats_stats', header="true")
    featureDF.unpersist() 

In [12]:
getWindows(timestampData.df, labelDF)

0.0021284747350048955
Processing started at:  1534523976 2018-08-17 09:39:36.254427
Time taken:  0
0.004256949470009791
Processing started at:  1534523979 2018-08-17 09:39:39.489485
Time taken:  0
0.0063854242050146864
Processing started at:  1534523982 2018-08-17 09:39:42.673045
Time taken:  0
0.008513898940019582
Processing started at:  1534523985 2018-08-17 09:39:45.719339
Time taken:  0
0.010642373675024477
Processing started at:  1534523988 2018-08-17 09:39:48.989822
Time taken:  0
0.012770848410029373
Processing started at:  1534523991 2018-08-17 09:39:51.961860
Time taken:  0
0.014899323145034268
Processing started at:  1534523994 2018-08-17 09:39:54.944251
Time taken:  0
0.017027797880039164
Processing started at:  1534523997 2018-08-17 09:39:57.935795
Time taken:  0
0.01915627261504406
Processing started at:  1534524000 2018-08-17 09:40:00.931902
Time taken:  0
0.021284747350048955
Processing started at:  1534524003 2018-08-17 09:40:03.931198
Time taken:  0
0.02341322208505385

Time taken:  0
0.183048827210421
Processing started at:  1534524247 2018-08-17 09:44:07.469023
Time taken:  0
0.18517730194542592
Processing started at:  1534524250 2018-08-17 09:44:10.884359
Time taken:  0
0.1873057766804308
Processing started at:  1534524254 2018-08-17 09:44:14.387835
Time taken:  0
0.18943425141543568
Processing started at:  1534524257 2018-08-17 09:44:17.903386
Time taken:  0
0.1915627261504406
Processing started at:  1534524261 2018-08-17 09:44:21.383159
Time taken:  0
0.1936912008854455
Processing started at:  1534524265 2018-08-17 09:44:25.009313
Time taken:  0
0.19581967562045036
Processing started at:  1534524268 2018-08-17 09:44:28.457275
Time taken:  0
0.19794815035545527
Processing started at:  1534524272 2018-08-17 09:44:32.018159
Time taken:  0
0.2000766250904602
Processing started at:  1534524275 2018-08-17 09:44:35.451218
Time taken:  0
0.20220509982546508
Processing started at:  1534524278 2018-08-17 09:44:38.918412
Time taken:  0
0.20433357456046997
P

Time taken:  0
0.36396917968583714
Processing started at:  1534524571 2018-08-17 09:49:31.662889
Time taken:  0
0.366097654420842
Processing started at:  1534524575 2018-08-17 09:49:35.774772
Time taken:  0
0.3682261291558469
Processing started at:  1534524580 2018-08-17 09:49:40.114114
Time taken:  0
0.37035460389085184
Processing started at:  1534524584 2018-08-17 09:49:44.181924
Time taken:  0
0.3724830786258567
Processing started at:  1534524588 2018-08-17 09:49:48.345994
Time taken:  0
0.3746115533608616
Processing started at:  1534524592 2018-08-17 09:49:52.476403
Time taken:  0
0.3767400280958665
Processing started at:  1534524596 2018-08-17 09:49:56.516402
Time taken:  0
0.37886850283087137
Processing started at:  1534524600 2018-08-17 09:50:00.592240
Time taken:  0
0.38099697756587625
Processing started at:  1534524604 2018-08-17 09:50:04.721857
Time taken:  0
0.3831254523008812
Processing started at:  1534524609 2018-08-17 09:50:09.155556
Time taken:  0
0.3852539270358861
Pro

0.5448895321612532
Processing started at:  1534524962 2018-08-17 09:56:02.219782
Time taken:  0
0.5470180068962581
Processing started at:  1534524966 2018-08-17 09:56:06.948932
Time taken:  0
0.549146481631263
Processing started at:  1534524971 2018-08-17 09:56:11.720709
Time taken:  0
0.5512749563662679
Processing started at:  1534524976 2018-08-17 09:56:16.549753
Time taken:  0
0.5534034311012728
Processing started at:  1534524981 2018-08-17 09:56:21.326080
Time taken:  0
0.5555319058362777
Processing started at:  1534524986 2018-08-17 09:56:26.101354
Time taken:  0
0.5576603805712826
Processing started at:  1534524991 2018-08-17 09:56:31.166841
Time taken:  0
0.5597888553062875
Processing started at:  1534524995 2018-08-17 09:56:35.960137
Time taken:  0
0.5619173300412924
Processing started at:  1534525000 2018-08-17 09:56:40.718026
Time taken:  0
0.5640458047762973
Processing started at:  1534525005 2018-08-17 09:56:45.549805
Time taken:  0
0.5661742795113023
Processing started at:

Time taken:  0
0.7279383593716743
Processing started at:  1534525413 2018-08-17 10:03:33.722860
Time taken:  0
0.7300668341066792
Processing started at:  1534525419 2018-08-17 10:03:39.570666
Time taken:  0
0.732195308841684
Processing started at:  1534525425 2018-08-17 10:03:45.197659
Time taken:  0
0.7343237835766889
Processing started at:  1534525430 2018-08-17 10:03:50.630602
Time taken:  0
0.7364522583116938
Processing started at:  1534525436 2018-08-17 10:03:56.133108
Time taken:  0
0.7385807330466987
Processing started at:  1534525441 2018-08-17 10:04:01.690618
Time taken:  0
0.7407092077817037
Processing started at:  1534525447 2018-08-17 10:04:07.207717
Time taken:  0
0.7428376825167086
Processing started at:  1534525452 2018-08-17 10:04:12.837389
Time taken:  0
0.7449661572517134
Processing started at:  1534525458 2018-08-17 10:04:18.638032
Time taken:  0
0.7470946319867183
Processing started at:  1534525464 2018-08-17 10:04:24.159571
Time taken:  0
0.7492231067217232
Process

...

Processing started at:  1534554498 2018-08-17 18:08:18.529079
Time taken:  0
4.576220680260525
Processing started at:  1534554546 2018-08-17 18:09:06.743376
Time taken:  0
4.5783491549955295
Processing started at:  1534554594 2018-08-17 18:09:54.966858
Time taken:  0
4.580477629730535
Processing started at:  1534554643 2018-08-17 18:10:43.255381
Time taken:  0
4.5826061044655395
Processing started at:  1534554691 2018-08-17 18:11:31.604332
Time taken:  0
4.584734579200545
Processing started at:  1534554739 2018-08-17 18:12:19.963761
Time taken:  0
4.5868630539355495
Processing started at:  1534554788 2018-08-17 18:13:08.300594
Time taken:  0
4.5889915286705545
Processing started at:  1534554839 2018-08-17 18:13:59.598584
Time taken:  0
4.5911200034055595
Processing started at:  1534554887 2018-08-17 18:14:47.289879
Time taken:  0
4.5932484781405645
Processing started at:  1534554935 2018-08-17 18:15:35.412785
Time taken:  0
4.5953769528755695
Processing started at:  1534554983 2018-08-

Time taken:  0
4.759269507470947
Processing started at:  1534559224 2018-08-17 19:27:04.693946
Time taken:  0
4.761397982205951
Processing started at:  1534559285 2018-08-17 19:28:05.509865
Time taken:  3
4.763526456940956
Processing started at:  1534559349 2018-08-17 19:29:09.361437
Time taken:  0
4.765654931675961
Processing started at:  1534559410 2018-08-17 19:30:10.216426
Time taken:  0
4.767783406410966
Processing started at:  1534559470 2018-08-17 19:31:10.944893
Time taken:  0
4.769911881145971
Processing started at:  1534559531 2018-08-17 19:32:11.579767
Time taken:  3
4.772040355880976
Processing started at:  1534559595 2018-08-17 19:33:15.067430
Time taken:  0
4.77416883061598
Processing started at:  1534559658 2018-08-17 19:34:18.380417
Time taken:  0
4.776297305350986
Processing started at:  1534559722 2018-08-17 19:35:22.037517
Time taken:  0
4.77842578008599
Processing started at:  1534559785 2018-08-17 19:36:25.897256
Time taken:  0
4.780554254820996
Processing started 

Time taken:  0
4.944446809416372
Processing started at:  1534565465 2018-08-17 21:11:05.959466
Time taken:  0
4.946575284151377
Processing started at:  1534565552 2018-08-17 21:12:32.433887
Time taken:  0
4.948703758886382
Processing started at:  1534565641 2018-08-17 21:14:01.270279
Time taken:  0
4.950832233621387
Processing started at:  1534565726 2018-08-17 21:15:26.476616
Time taken:  0
4.952960708356391
Processing started at:  1534565811 2018-08-17 21:16:51.997382
Time taken:  3
4.955089183091397
Processing started at:  1534565898 2018-08-17 21:18:18.351249
Time taken:  0
4.957217657826401
Processing started at:  1534565984 2018-08-17 21:19:44.577404
Time taken:  0
4.959346132561406
Processing started at:  1534566070 2018-08-17 21:21:10.632541
Time taken:  0
4.961474607296411
Processing started at:  1534566155 2018-08-17 21:22:35.854478
Time taken:  3
4.963603082031416
Processing started at:  1534566244 2018-08-17 21:24:04.651402
Time taken:  0
4.965731556766421
Processing starte

Processing started at:  1534575713 2018-08-18 00:01:53.533667
Time taken:  0
5.129624111361799
Processing started at:  1534575903 2018-08-18 00:05:03.497118
Time taken:  0
5.131752586096803
Processing started at:  1534576101 2018-08-18 00:08:21.242413
Time taken:  3
5.133881060831809
Processing started at:  1534576316 2018-08-18 00:11:56.805395
Time taken:  3
5.136009535566813
Processing started at:  1534576563 2018-08-18 00:16:03.720906
Time taken:  0
5.138138010301818
Processing started at:  1534576869 2018-08-18 00:21:09.892315
Time taken:  0


Py4JJavaError: An error occurred while calling o4408636.union.
: java.lang.OutOfMemoryError: Java heap space
